In [3]:
import pandas as pd
from pathlib import Path

In [12]:
train_data_path = Path('.').resolve().parent / 'data' / 'fraudTrain.csv'
test_data_path = Path('.').resolve().parent / 'data' / 'fraudTest.csv'

train_data = pd.read_csv(train_data_path, index_col=0)
test_data = pd.read_csv(test_data_path, index_col=0)

# EDA

same features?

In [21]:
sorted(train_data.columns) == sorted(test_data.columns)

True

same dtypes?

In [25]:
all((train_data.dtypes == test_data.dtypes))

True

duplicated?

In [31]:
train_data.duplicated().any(), test_data.duplicated().any()

(np.False_, np.False_)

null?

In [36]:
train_data.isna().any().any(), test_data.isna().any().any()

(np.False_, np.False_)

In [43]:
train_data.dtypes

trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object